In [40]:
import numpy as num

#print vector to check final solution's rref
def print_vector(vector):
    for item in vector:
        print (item)

#print intermediate boards
def print_matrices(M1, M2):
    print('\t'*int(len(M1)/2)+"\t"*len(M1))
    for i in range(len(M1)):
        #use formatting placeholders for neat output of elements
        row1 = ['{0:+7.3f}'.format(x) for x in M1[i]]
        row2 = ['{0:+7.3f}'.format(x) for x in M2[i]]
        print(row1,'\t', row2)

#make a copy of the board for computations
def duplicate_matrix(M):
    rows = len(M)
    cols = len(M[0])
    DP = num.zeros([rows, cols], dtype = int) 
    #fill in values from M into DP
    for i in range(rows):
        for j in range(cols):
            DP[i][j] = M[i][j]
    return DP

#Multiply given matrices A and B
def matrix_multiply(A,B):
    rowsA = len(A)
    colsA = len(A[0])
    rowsB = len(B)
    colsB = len(B[0])
    C = num.zeros([rowsA, colsB], dtype = int)
    #Compute the products by traversing the rows and columns of each matrix
    for i in range(rowsA):
        for j in range(colsB):
            total = 0
            for h in range(colsA):
                total += A[i][h] * B[h][j]
            C[i][j] = total
    return C

In [41]:
# A = [[5.,4.,3.,2.,1.],[4.,3.,2.,1.,5.],[3.,2.,9.,5.,4.],[2.,1.,5.,4.,3.],[1.,2.,3.,4.,5.]]
# B = [[48],[38],[64],[43],[42]]
A = [[5.,3.,1.],[3.,9.,4.],[1.,3.,5.]]
B = [[9.0],[16.0],[9.0]]

In [42]:
AM = duplicate_matrix(A)
n = len(A)
BM = duplicate_matrix(B)

print()

#Create a list that allows for flexible number of squares
squares = list(range(n))
for diag in range(n):
    scalar = 1.0 / AM[diag][diag]
    
    #Multiply by scalar for every column for the diagonal row
    for j in range(n):
        AM[diag][j] *= scalar
    BM[diag][0] *= scalar    
    
    #Print out current matrix after computation
    print_matrices(AM, BM)
    print()
    
    #Iterate through and scale all rows except the fd row
    for i in squares[0:diag] + squares[diag+1:]:
        currRowScalar = AM[i][diag]
        #compute current row - current row scaler*fdRow for every column
        for j in range(n):
            AM[i][j] = AM[i][j] - currRowScalar * AM[diag][j]
        BM[i][0] = BM[i][0] - currRowScalar * BM[diag][0]
        
        # Print out current matrix after computation
        print_matrices(AM, BM)
        print()


				
[' +1.000', ' +0.000', ' +0.000'] 	 [' +1.000']
[' +3.000', ' +9.000', ' +4.000'] 	 ['+16.000']
[' +1.000', ' +3.000', ' +5.000'] 	 [' +9.000']

				
[' +1.000', ' +0.000', ' +0.000'] 	 [' +1.000']
[' +0.000', ' +9.000', ' +4.000'] 	 ['+13.000']
[' +1.000', ' +3.000', ' +5.000'] 	 [' +9.000']

				
[' +1.000', ' +0.000', ' +0.000'] 	 [' +1.000']
[' +0.000', ' +9.000', ' +4.000'] 	 ['+13.000']
[' +0.000', ' +3.000', ' +5.000'] 	 [' +8.000']

				
[' +1.000', ' +0.000', ' +0.000'] 	 [' +1.000']
[' +0.000', ' +1.000', ' +0.000'] 	 [' +1.000']
[' +0.000', ' +3.000', ' +5.000'] 	 [' +8.000']

				
[' +1.000', ' +0.000', ' +0.000'] 	 [' +1.000']
[' +0.000', ' +1.000', ' +0.000'] 	 [' +1.000']
[' +0.000', ' +3.000', ' +5.000'] 	 [' +8.000']

				
[' +1.000', ' +0.000', ' +0.000'] 	 [' +1.000']
[' +0.000', ' +1.000', ' +0.000'] 	 [' +1.000']
[' +0.000', ' +0.000', ' +5.000'] 	 [' +5.000']

				
[' +1.000', ' +0.000', ' +0.000'] 	 [' +1.000']
[' +0.000', ' +1.000', ' +0.000'] 	 [' +1.000'

In [43]:
print("Now we multiply the original A matrix times our solution for X, which is BM. If we get our starting X, our solution is proved.")
print_vector(matrix_multiply(A,BM))

Now we multiply the original A matrix times our solution for X, which is BM. If we get our starting X, our solution is proved.
[9]
[16]
[9]


In [ ]:
#insert values in agent board using inferences calculated from system of equations
for item in BM:
    agent_board[item[0][0]][item[0][1]] = item[1]
print(agent_board)